In [1]:
import torch
from torch.utils.cpp_extension import load
import os

os.environ['CUDA_LAUNCH_BLOCKING']="1"

torch integer data flow 는 gpu에서 사용 불가

In [2]:
# int_data = torch.randint(0,255,(1,3,24,24), dtype=torch.uint8)
# weight = torch.randint(0,255,(1,3,3,3), dtype=torch.uint8)

# # b = torch.nn.functional.conv2d(int_data.cuda(), weight=weight.cuda(),stride=1)
# b = torch.nn.functional.conv2d(int_data, weight=weight,stride=1,bias=None, padding=1, dtype=torch.uint8)
# print(b.shape)

In [3]:
import int8mm_cuda
from torch.nn.modules import Module

class Intmm(Module):
    def __init__(self, in_channels, out_channels):
        super(Intmm,self).__init__()
        self.weight = torch.randint(-127,127,(out_channels, in_channels), dtype=torch.int8).cuda()

    def forward(self,x):
        # weight [OUT, IN} - > [IN, OUT]
        # input [BATCH, IN]
        return int8mm_cuda.int8_mm(x,self.weight.transpose(1,0).contiguous())

mm = Intmm(4,12)
x = torch.randint(-127,127,(1,4), dtype=torch.int8).cuda()
print(x.dtype, mm.weight.dtype)
with torch.no_grad():
    y = mm(x)
print(x)
print(mm.weight)
print(y)

torch.int8 torch.int8
tensor([[ 68, 126,  65, -78]], device='cuda:0', dtype=torch.int8)
tensor([[  -7,  -46,  -17,   62],
        [ -87,   77,   77,  -17],
        [ -56,   77,   81, -115],
        [ 117,  -51,   55,   97],
        [ -63,  -23,  -63,  119],
        [ 126,  -25,   84,  115],
        [  82,   59,  114,  -42],
        [  83,   52, -127,  102],
        [-110,   72,  -94,   11],
        [-113,  100,   67, -110],
        [  28,    1,  -89,   45],
        [ -50,  -90,    1,    3]], device='cuda:0', dtype=torch.int8)
tensor([[-12213,  10117,  20129,  -2461, -20559,   1908,  23696,  -4015,  -5376,
          17851,  -7265, -14909]], device='cuda:0', dtype=torch.int32)


In [4]:
import numpy as np 
print(mm.weight.transpose(1,0),end="\n\n")
x_data = x.detach().cpu().numpy()
mm_data = mm.weight.detach().cpu().numpy()
print(f"x data - {x_data}\n")
print(f"mm data - {mm_data}\nmm Trans - {mm_data.T}\n")
y = x_data @ mm_data.T
print(f"y - {y}")

tensor([[  -7,  -87,  -56,  117,  -63,  126,   82,   83, -110, -113,   28,  -50],
        [ -46,   77,   77,  -51,  -23,  -25,   59,   52,   72,  100,    1,  -90],
        [ -17,   77,   81,   55,  -63,   84,  114, -127,  -94,   67,  -89,    1],
        [  62,  -17, -115,   97,  119,  115,  -42,  102,   11, -110,   45,    3]],
       device='cuda:0', dtype=torch.int8)

x data - [[ 68 126  65 -78]]

mm data - [[  -7  -46  -17   62]
 [ -87   77   77  -17]
 [ -56   77   81 -115]
 [ 117  -51   55   97]
 [ -63  -23  -63  119]
 [ 126  -25   84  115]
 [  82   59  114  -42]
 [  83   52 -127  102]
 [-110   72  -94   11]
 [-113  100   67 -110]
 [  28    1  -89   45]
 [ -50  -90    1    3]]
mm Trans - [[  -7  -87  -56  117  -63  126   82   83 -110 -113   28  -50]
 [ -46   77   77  -51  -23  -25   59   52   72  100    1  -90]
 [ -17   77   81   55  -63   84  114 -127  -94   67  -89    1]
 [  62  -17 -115   97  119  115  -42  102   11 -110   45    3]]

y - [[  75 -123  -95   99  -79  116 -112   81 

Pooling layer


pytorch conv layer parmeter shape 보기

In [5]:
import torchvision
model = torchvision.models.vgg.vgg16(pretrained=True)
print(model.features[0].weight.shape)
c = torch.nn.Conv2d(4,12,3,1,1)
print(c.weight.shape) # NCHW

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([64, 3, 3, 3])
torch.Size([12, 4, 3, 3])


In [6]:
import cutlassconv
from torch.nn.modules import Module

class IntConv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride =1, padding =1):
        super(IntConv2d,self).__init__()
        self.weight = torch.randint(0,127,(out_channels, kernel_size, kernel_size, in_channels), dtype=torch.int8).cuda()
        self.stride = stride
        self.padding = padding

    def forward(self,x):
        # trans_weight = torch.flip(self.weight,[1,2]).transpose(0,3).contiguous()
        # trans_weight = self.weight.permute(0,2,3,1).contiguous()
        trans_weight = self.weight
        return cutlassconv.int8_conv(x,trans_weight)
## cutlass는 16의 배수만
input_channel= 16
conv = IntConv2d(input_channel,32,3,1,1)
print(conv.weight.shape)
x = torch.randint(0,127,(1,32,32,input_channel), dtype=torch.int8).cuda()


torch.Size([32, 3, 3, 16])


In [7]:
with torch.no_grad():
    y = conv(x)


In [8]:
import numpy as np 
conv_data = conv.weight.detach().cpu().numpy()
x_data = x.detach().cpu().numpy()
y_data = y.detach().cpu().numpy()
print(f"x data - {x.shape} \n{x_data}\n")
print(f"conv data - {conv_data.shape}\n{conv_data}\n")
print(f"y data - {y.shape}\n{y}")

x data - torch.Size([1, 32, 32, 16]) 
[[[[ 23  80  65 ...   8 112  62]
   [ 15 121 103 ... 106  97 126]
   [ 12 110  61 ...  91  60  61]
   ...
   [ 92 122  16 ...  38  51   2]
   [ 32  46  31 ...  97  63 113]
   [  2   7 103 ...  50  92  69]]

  [[ 24  72  59 ... 116   1  91]
   [ 94  52  15 ...  24  59  46]
   [ 81  25 102 ...  22 118  47]
   ...
   [ 55 111  17 ...  33 121 108]
   [119 108  23 ...  19  36 122]
   [112  84 120 ...  85  10  98]]

  [[ 58  36 106 ...  85 101 118]
   [119  83  57 ...  86  36  47]
   [ 79 121 107 ...  22  76 125]
   ...
   [120  32   5 ... 116  26  46]
   [ 49 114 124 ... 114  81 106]
   [123   8 116 ...  51  21  81]]

  ...

  [[ 89  96  32 ...  56  13  73]
   [ 74  74  68 ...   3  98  56]
   [ 12  47 118 ...  42  20   2]
   ...
   [114  57  86 ...   0  71  22]
   [ 19  13 122 ... 107  38  27]
   [107  10  83 ...  46  10  50]]

  [[117   6  26 ...  81  84  52]
   [ 64  92  40 ...  92  45   1]
   [ 23  11  80 ... 114  81  16]
   ...
   [ 83  67  26 ... 1

In [9]:
import int8conv_cuda
from torch.nn.modules import Module

class IntConv2d(Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride =1, padding =1):
        super(IntConv2d,self).__init__()
        self.weight = torch.randint(0,127,(out_channels, kernel_size, kernel_size, in_channels), dtype=torch.int8).cuda()
        self.stride = stride
        self.padding = padding

    def forward(self,x):
        # trans_weight = torch.flip(self.weight,[1,2]).transpose(0,3).contiguous()
        # trans_weight = self.weight.permute(0,2,3,1).contiguous()
        trans_weight = self.weight

        return int8conv_cuda.cu_int8_conv(x,trans_weight,self.stride, self.padding,1)
    
# cudnn은 4의 배수만
input_channel= 4
conv = IntConv2d(input_channel,32,3,1,1)
print(conv.weight.shape)
x = torch.randint(0,127,(1,32,32,input_channel), dtype=torch.int8).cuda()


torch.Size([32, 3, 3, 4])


In [10]:
with torch.no_grad():
    y = conv(x)
print(y.shape)

torch.Size([1, 32, 32, 32])


In [11]:
import numpy as np 
conv_data = conv.weight.detach().cpu().numpy()
x_data = x.detach().cpu().numpy()
y_data = y.detach().cpu().numpy()
print(f"x data - {x.shape} \n{x_data}\n")
print(f"conv data - {conv_data.shape}\n{conv_data}\n")
print(f"y data - {y.shape}\n{y}")

x data - torch.Size([1, 32, 32, 4]) 
[[[[120 109  31  86]
   [125  39 113  78]
   [ 58  96  35 102]
   ...
   [117  81  63  24]
   [ 24  36 116  32]
   [ 94  47  38  54]]

  [[118  48  75  75]
   [ 99  91  47   6]
   [ 85 105  14  15]
   ...
   [ 54  13 118  47]
   [ 97  94  87   6]
   [ 12  37  61   1]]

  [[ 34  64  24  98]
   [ 13 103   4 109]
   [ 51 123  24  35]
   ...
   [ 64   7   1  74]
   [102  63  52  41]
   [ 45  53  50  83]]

  ...

  [[ 60  36 100  60]
   [ 70  84  15  23]
   [106 110  77  79]
   ...
   [ 96   1  86  76]
   [ 93 121  43 111]
   [ 13  50  43  97]]

  [[ 49 103 108 116]
   [ 72  31  20  74]
   [ 47   7  60  72]
   ...
   [ 92   3   9  88]
   [123 102   4  41]
   [ 12 124  51  97]]

  [[121   0   8 120]
   [117  76  44  87]
   [ 67  13  52  70]
   ...
   [ 51 102   0 114]
   [ 16  48  72   7]
   [ 90  91  81  61]]]]

conv data - (32, 3, 3, 4)
[[[[ 84  31  82  18]
   [ 37  12 123  71]
   [ 75  75  87 104]]

  [[ 77 119  27 115]
   [ 22  81  74 118]
   [ 15 109